In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
import argparse
import glob
import json
import os
from pathlib import Path

import numpy as np
import torch
import yaml
from tqdm import tqdm

from utils.google_utils import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, box_iou, \
    non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, clip_coords, set_logging, increment_path
from utils.loss import compute_loss
from utils.metrics import ap_per_class
from utils.plots import plot_images, output_to_target
from utils.torch_utils import select_device, time_synchronized

from models.models import *

ModuleNotFoundError: No module named 'utils'

In [ ]:
def load_classes(path):
    # Loads *.names file at 'path'
    with open(path, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))  # filter removes empty strings (such as last line)

In [11]:
data = "../zero-waste-1/data.yaml"
weights="../runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt",
batch_size=16,
imgsz=640,
conf_thres=0.5,
iou_thres=0.6,  # for NMS
save_json=True,
single_cls=False,
augment=False,
verbose=True,
model=None,
dataloader=None,
save_dir=Path(''),  # for saving images
save_txt=False,  # for auto-labelling
save_conf=False,
plots=True,
log_imgs=0  # number of logged images
task = "test"

In [34]:
from argparse import Namespace
opt = Namespace(augment=False, batch_size=32, cfg='./cfg/yolor_p6.cfg', conf_thres=0.5, data='../zero-waste-1/data.yaml', device='cpu', exist_ok=False, img_size=416, iou_thres=0.65, name='exp', names='./data/zerowaste.names', project='../runs/test', save_conf=True, save_json=True, save_txt=False, single_cls=False, task='test', verbose=True, weights=['../runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt'])

In [53]:
augment=False
batch_size=32
cfg='cfg/yolor_p6.cfg'
conf_thres=0.5
data='../zero-waste-1/data.yaml'
device='cpu'
exist_ok=False
img_size=416
iou_thres=0.65
name='exp'
names='data/zerowaste.names'
project='../runs/test'
save_conf=True
save_json=True
save_txt=False
single_cls=False
task='test'
verbose=True
weights=['../runs/train/yolor_p6_2022_04_05-19_58_08/weights/best_overall.pt']

In [54]:
set_logging()
device = select_device(opt.device, batch_size=batch_size)
save_txt = opt.save_txt  # save *.txt labels

# Directories: where all the logging file will be saved for this experiment
save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

# Load model
model = Darknet(opt.cfg).to(device)
try:
    ckpt = torch.load(weights[0], map_location=device)  # load checkpoint
    ckpt['model'] = {k: v for k, v in ckpt['model'].items() if model.state_dict()[k].numel() == v.numel()}
    model.load_state_dict(ckpt['model'], strict=False)
except:
    load_darknet_weights(model, weights[0])
imgsz = check_img_size(imgsz, s=64)  # check img_size

# Half: half precision if we are running on GPU
half = device.type != 'cpu'
if half:
    model.half()



Using torch 1.9.0+cu111 CPU

Model Summary: 665 layers, 36854616 parameters, 36854616 gradients, 80.410315000 GFLOPS


In [55]:
print(data)

../zero-waste-1/data.yaml


In [56]:
# Configure
model.eval()
is_coco = data.endswith('coco.yaml')  # is COCO dataset
with open(data) as f:
    data = yaml.load(f, Loader=yaml.FullLoader)  # model dict
check_dataset(data)  # check
nc = 1 if single_cls else int(data['nc'])  # number of classes
iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

# Logging
log_imgs, wandb = min(log_imgs, 100), None  # ceil
try:
    import wandb  # Weights & Biases
except ImportError:
    log_imgs = 0

# Dataloader
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
path = data['test'] if task == 'test' else data['val']  # path to val/test images
dataloader = create_dataloader(path, imgsz, batch_size, 64, opt, pad=0.5, rect=True)[0]

Exception: Dataset not found.

In [3]:
#initialize COCO ground truth api
dataDir='../'
dataType='val2014'
annFile = '%s/annotations/%s_%s.json'%(dataDir,prefix,dataType)
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=8.01s)
creating index...
index created!


In [4]:
#initialize COCO detections api
resFile='%s/results/%s_%s_fake%s100_results.json'
resFile = resFile%(dataDir, prefix, dataType, annType)
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...     
DONE (t=0.05s)
creating index...
index created!


In [5]:
imgIds=sorted(cocoGt.getImgIds())
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

In [6]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...      
DONE (t=0.46s).
Accumulating evaluation results...   
DONE (t=0.38s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.697
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.573
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.586
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.501
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.387
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.640
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.566
 Average Rec